In [0]:
import numpy as np 
import pandas as pd
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

In [63]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [64]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/final.csv')
data = data[['sentiment','content']]	
data['content'].head(), data['sentiment'].head()

(0    @tiffanylue i know  i was listenin to bad habi...
 1    Layin n bed with a headache  ughhhh...waitin o...
 2                  Funeral ceremony...gloomy friday...
 3    @dannycastillo We want to trade with someone w...
 4    Re-pinging @ghostridah14: why didn't you go to...
 Name: content, dtype: object, 0      boredom
 1    depressed
 2    depressed
 3      neutral
 4    depressed
 Name: sentiment, dtype: object)

In [0]:
max_fatures = 100
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['content'].values)
X = tokenizer.texts_to_sequences(data['content'].values)
X = pad_sequences(X)
embed_dim = 128
lstm_out = 196

#embed_dim = 200
#lstm_out = 350

In [66]:
X

array([[ 0,  0,  0, ...,  6,  1, 24],
       [ 0,  0,  0, ...,  4, 14, 43],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  9, 43, 20],
       [ 0,  0,  0, ..., 53, 10,  3],
       [ 0,  0,  0, ...,  6,  1, 15]], dtype=int32)

In [67]:
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(6,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 25, 128)           12800     
_________________________________________________________________
spatial_dropout1d_4 (Spatial (None, 25, 128)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 1182      
Total params: 268,782
Trainable params: 268,782
Non-trainable params: 0
_________________________________________________________________
None


In [68]:
Y = pd.get_dummies(data['sentiment']).values
Y.shape

(35528, 6)

In [56]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
batch_size = 32
model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2)

validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
#print("score: %.2f" % (score))
print("accuracy: %.2f" % (acc))

(23803, 25) (23803, 6)
(11725, 25) (11725, 6)
Epoch 1/5
 - 51s - loss: 1.4403 - acc: 0.4286
Epoch 2/5
 - 50s - loss: 1.3903 - acc: 0.4524
Epoch 3/5
 - 49s - loss: 1.3813 - acc: 0.4573
Epoch 4/5
 - 50s - loss: 1.3766 - acc: 0.4605
Epoch 5/5
 - 50s - loss: 1.3719 - acc: 0.4631
accuracy: 0.46


In [57]:
Y[0], X[0]

(array([0, 1, 0, 0, 0, 0], dtype=uint8),
 array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         1, 57,  1, 26,  2,  6,  1, 24], dtype=int32))

In [59]:
twt = "i am sad"

twt = tokenizer.texts_to_sequences(twt)

twt = pad_sequences(twt, maxlen=25, dtype='int32', padding='post', truncating='post', value=0)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
print(sentiment)
print(np.argmax(sentiment))
if(np.argmax(sentiment) == 0):
    print("anger")
elif (np.argmax(sentiment) == 1):
    print("neutral")
elif (np.argmax(sentiment) == 2):
    print("happiness")
elif (np.argmax(sentiment) == 3):
    print("boredom")
elif (np.argmax(sentiment) == 4):
    print("love")
elif (np.argmax(sentiment) == 5):
    print("depressed")

[0.03944729 0.06311712 0.21106036 0.16032204 0.06976652 0.45628658]
5
sad
